# Lab 3 - The Matrix Form and The Duality Theory

<b>Information on group members:</b><br>
1) 154974, Andrii Chmutov <br>
2) 158669, Vasyl Korzavatykh

In [1]:
import itertools

import numpy as np
import pandas as pd
from IPython.display import display
from pulp import LpConstraint, LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum

# 1) The Matrix Form - Fundamental Insight (finish this part to get 3.0)

1.1) Given is the below (primal) linear problem:

primal problem:<br>
MAXIMIZE<br>
4*x1 + 2*x2 + 3*x3 <br>

SUBJECT TO<br>
1_constraint: x1 + x2 <= 10<br>
2_constraint: 2*x2 + x3 <= 12<br>
3_constraint: 3*x1 + 2*x3 <= 15<br>
4_constraint: x1 + x2 + x3 <= 20<br>

VARIABLES<br>
x1 Continuous<br>
x2 Continuous<br>
x3 Continuous<br>

1.2) Use the PuLP library to solve the above problem. Identify the optimal solution: the values for basic variables and the corresponding value for the objective function.

In [2]:
# Model here:
model = LpProblem(name="problem", sense=LpMaximize)
var_names = ["1", "2", "3"]
x = LpVariable.dicts("x", var_names, 0)

const_names = ["1", "2", "3", "4"]
sense = [-1, -1, -1, -1]
coefs = [[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]]
rhs = [10, 12, 15, 20]

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(3)])
    model += LpConstraint(e=expr, sense=s, name=cn, rhs=r)

obj_coefs = [4, 2, 3]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(3)])

model

problem:
MAXIMIZE
4*x_1 + 2*x_2 + 3*x_3 + 0
SUBJECT TO
1: x_1 + x_2 <= 10

2: 2 x_2 + x_3 <= 12

3: 3 x_1 + 2 x_3 <= 15

4: x_1 + x_2 + x_3 <= 20

VARIABLES
x_1 Continuous
x_2 Continuous
x_3 Continuous

In [3]:
# Solve here
model.solve()
for n in var_names:
    print(f"x{n}: {x[n].value()}")
print("Objective:", model.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/warlock/prog/venvs/tf/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/3f56de3288de453f8bb13fc70ec081a7-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/3f56de3288de453f8bb13fc70ec081a7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 9 COLUMNS
At line 22 RHS
At line 27 BOUNDS
At line 28 ENDATA
Problem MODEL has 4 rows, 3 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (-1) rows, 3 (0) columns and 6 (-3) elements
0  Obj -0 Dual inf 10.499997 (3)
3  Obj 31.428571
Optimal - objective value 31.428571
After Postsolve, objective 31.428571, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 31.42857143 - 3 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallcloc

1.3) In this exercise, you are asked to identify ALL basic (feasible and not) solutions to the above problem. We will do this naively. Specifically, you are asked to use the fundamental insight to build a final simplex tableau for each possible base. Therefore, you need first to initialize the data: c, b, A matrixes, and it is suggested to initialize the auxiliary matrix M defined as M = A + (concatenate) I (identity matrix). Note that the problem should be formulated in the augmented form. Then, you have to iterate over each possible base B, compute B-1, and other relevant parts for the simplex tableau.<br><br>
a) Identify the optimal solution using the optimality condition; print it (Z value and values for basic variables); compare thus derived solution with the optimum found using the PuLP library (obviously, both solutions should be the same). <br>
b) Count the number of feasible and infeasible solutions. How many (all) basic solutions to the problem can be identified? <br><br>
It is suggested to use the NumPy library for performing matrix operations. 

In [4]:
# Initialize the data here:
c = np.array([4, 2, 3, 0, 0, 0, 0])
b = np.array([10, 12, 15, 20])
A = np.array([[1, 1, 0], [0, 2, 1], [3, 0, 2], [1, 1, 1]])
M = np.concatenate((A, np.identity(4)), axis=1)

<b> Important note: the below is just a proposition. You can solve the problem in your own way. </b>

You can define an auxiliary method constructing a final simplex tableau for a given base.  Here, "base" is a list of columns (integers) for the base. Note that the functions in python can return multiple objects and you can use this functionality to return<br>
- the inversed base<br>
- coefficients in the row 0 for slack variables<br>
- right side values (except the objective function value)<br>
- the objective function value<br>
- the coefficients for decision variables in row 0 <br>
- the coefficients for decision variables in rows 1+<br>

Note that if BI cannot be built (it is possible), the method may return None in order to notify the executive method about this exception. 



In [5]:
def getFinalTableau(
    base: np.ndarray, c: np.ndarray, b: np.ndarray, A: np.ndarray, M: np.ndarray
) -> dict:
    B = M[:, base]

    try:
        B_inv = np.linalg.inv(B)
    except Exception:
        return None

    c_B = c[base]
    sc = c_B @ B_inv
    rs = B_inv @ b
    obj = c_B @ rs
    dc_0 = sc @ A - c[: len(c[c > 0])]
    dc_1 = B_inv @ A

    rs_list = list(rs)
    rs_full = list(base)
    for i in range(len(rs_full)):
        if rs_full[i]:
            rs_full[i] = rs_list.pop(0)
        else:
            rs_full[i] = 0

    return {
        "B_inv": B_inv,
        "c_B": c_B,
        "sc": sc,
        "rs": np.array(rs_full),
        "obj": obj,
        "dc_0": dc_0,
        "dc_1": dc_1,
        "base": base,
    }

In [6]:
# Solve the problem here
solutions = []

for ones_positions in itertools.combinations(range(7), 4):
    base = np.full(7, False)
    base[np.array(ones_positions)] = True

    res = getFinalTableau(base, c, b, A, M)
    if res is not None:
        solutions.append(res)

solutions.sort(key=lambda x: -x["obj"])
feasible_solutions = list(
    filter(lambda x: (np.array(x["rs"]) >= 0).all(), solutions)
)  # feasible solutions have RS >= 0

print("Optimal solution:")

for i in range(7):
    if feasible_solutions[0]["rs"][i]:
        print(f"x{i + 1}: {feasible_solutions[0]['rs'][i]}")
print(f"Objective: {feasible_solutions[0]['obj']}")

print(f"\nNumber of feasible solutions: {len(feasible_solutions)}")
print(f"Number of infeasible solutions: {len(solutions) - len(feasible_solutions)}")
print(f"Number of all solutions: {len(solutions)}")

Optimal solution:
x1: 4.428571428571427
x2: 5.571428571428572
x3: 0.8571428571428559
x7: 9.142857142857144
Objective: 31.428571428571423

Number of feasible solutions: 8
Number of infeasible solutions: 23
Number of all solutions: 31


## 2) The Duality Theory (finish this part + part 1 + to get 5.0)

2.1) Model the dual problem to the above solved primal one, using the PuLP library. Then, solve it and compare the derived optimum with the optimum for the primal problem. Are they equal?

In [7]:
# Model here:
model = LpProblem(name="dual_problem", sense=LpMinimize)
var_names = ["1", "2", "3", "4"]
x = LpVariable.dicts("y", var_names, 0)

const_names = ["1", "2", "3"]
sense = [1, 1, 1]
coefs = [[1, 0, 3, 1], [1, 2, 0, 1], [0, 1, 2, 1]]
rhs = [4, 2, 3]

for c, s, r, cn in zip(coefs, sense, rhs, const_names):
    expr = lpSum([x[var_names[i]] * c[i] for i in range(4)])
    model += LpConstraint(e=expr, sense=s, name=cn, rhs=r)

obj_coefs = [10, 12, 15, 20]
model += lpSum([x[var_names[i]] * obj_coefs[i] for i in range(4)])

model

dual_problem:
MINIMIZE
10*y_1 + 12*y_2 + 15*y_3 + 20*y_4 + 0
SUBJECT TO
1: y_1 + 3 y_3 + y_4 >= 4

2: y_1 + 2 y_2 + y_4 >= 2

3: y_2 + 2 y_3 + y_4 >= 3

VARIABLES
y_1 Continuous
y_2 Continuous
y_3 Continuous
y_4 Continuous

In [8]:
# Solve here:
model.solve()
for n in var_names:
    print(f"y{n}: {x[n].value()}")
print("Objective:", model.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/warlock/prog/venvs/tf/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/236fbb1958ab41e1af14c51396fe130a-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/236fbb1958ab41e1af14c51396fe130a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 8 COLUMNS
At line 22 RHS
At line 26 BOUNDS
At line 27 ENDATA
Problem MODEL has 3 rows, 4 columns and 9 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 3 (0) rows, 4 (0) columns and 9 (0) elements
0  Obj 0 Primal inf 3.833333 (3)
3  Obj 31.428571
Optimal - objective value 31.428571
Optimal objective 31.42857143 - 3 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

y1: 0.57142857
y2: 0.71428571
y3: 1.1428571
y4: 0.0
Objective: 31.42857072


2.2) This exercise is based on the exercise 1.3 (copy & paste solution). Here, you are asked to iterate over all basic solutions (as in 1.3) and store them along with their complementary dual solutions. Solutions should be stored in the PRIMAL_DUAL_SOLUTIONS list and sorted according to the objective value Z = W. Analyze their optimality and feasibility. Finally, you have to display all basic solutions wlong with their complementary solutions (you can use the provided piece of code written using the pandas library). <br><br>

PRIMAL_DUAL_SOLUTIONS is defined as a table consisting of n rows, where n is the number of basic solutions to the problem, and 21 columns. The columns are defined as follows:<br>
Col. 1: The objective value Z<br>
Col. 2-4: The values for decision variables (primal solution)<br>
Col. 5-8: The values for slack variables (primal solution)<br>
Col. 9: P_F = Y or N, Y/N = primal solution is feasible/infeasible<br>
Col. 10: P_O = Y or N, Y/N = primal solution is optimal/is not optimal<br>
Col. 11: P_STATE = -/suboptimal/superoptimal/optimal; depends on P_F and P_O (primal)<br>
Col. 12: D_STATE = -/suboptimal/superoptimal/optimal; depends on D_F and D_O (dual)<br>
Col. 13: D_F = Y or N, Y/N = dual solution is feasible/infeasible<br>
Col. 14: D_O = Y or N, Y/N = dual solution is optimal/is not optimal<br>
Col. 15-18: The values for decision variables (dual solution)<br>
Col. 19-21: The values for surplus variables (dual solution)<br><br>

Reminder: sort solutions according to Z; analyze how their states change with the increase of Z.

In [9]:
c = np.array([10, 12, 15, 20, 0, 0, 0])
b = np.array([4, 2, 3])
A = np.array([[1, 0, 3, 1], [1, 2, 0, 1], [0, 1, 2, 1]])
M = np.concatenate((A, -np.identity(3)), axis=1)

In [10]:
PRIMAL_DUAL_SOLUTIONS = []
tolerance = 10 ** (-5)

for primal in solutions[::-1]:
    dual_base = (
        np.concatenate((primal["base"][3:], primal["base"][:3])).astype(int) ^ 1
    ).astype(bool)
    dual = getFinalTableau(dual_base, c, b, A, M)

    assert abs(primal["obj"] - dual["obj"]) < tolerance  # Z == W

    primal_feasible = "Y" if (primal["rs"] >= 0).all() else "N"
    dual_feasible = "Y" if (dual["rs"] >= 0).all() else "N"
    primal_state, dual_state = "-", "-"

    if primal_feasible == "Y" and dual_feasible == "N":
        primal_state, dual_state = "suboptimal", "superoptimal"
    elif primal_feasible == "Y" and dual_feasible == "Y":
        primal_state, dual_state = tuple(["optimal" for _ in range(2)])
    elif primal_feasible == "N" and dual_feasible == "Y":
        primal_state, dual_state = "superoptimal", "suboptimal"

    PRIMAL_DUAL_SOLUTIONS.append(
        [primal["obj"]]
        + list(primal["rs"][:3])
        + list(primal["rs"][3:])
        + [
            primal_feasible,
            dual_feasible,
            primal_state,
            dual_state,
            dual_feasible,
            primal_feasible,
        ]
        + list(dual["rs"][:4])
        + list(dual["rs"][4:])
    )

In [11]:
# DISPLAY STORED PAIRS OF SOLUTIONS
df = pd.DataFrame(
    PRIMAL_DUAL_SOLUTIONS,
    columns=[
        "Z",
        "x1",
        "x2",
        "x3",
        "slack1",
        "slack2",
        "slack3",
        "slack4",
        "P_F",
        "P_O",
        "P_STATE",
        "D_STATE",
        "D_F",
        "F_O",
        "y1",
        "y2",
        "y3",
        "y4",
        "sure1",
        "sure2",
        "sure3",
    ],
)

display(
    df.style.set_properties(**{
        "width": "230px",
        "max-width": "230px",
    })
)

,Z,x1,x2,x3,slack1,slack2,slack3,slack4,P_F,P_O,P_STATE,D_STATE,D_F,F_O,y1,y2,y3,y4,sure1,sure2,sure3
0,-4.000000,0.000000,10.000000,-8.000000,0.000000,0.000000,31.000000,18.000000,N,N,-,-,N,N,-4.000000,3.000000,0.000000,0.000000,-8.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,10.000000,12.000000,15.000000,20.000000,Y,N,suboptimal,superoptimal,N,Y,0.000000,0.000000,0.000000,0.000000,-4.000000,-2.000000,-3.000000
2,12.000000,0.000000,6.000000,0.000000,4.000000,0.000000,15.000000,14.000000,Y,N,suboptimal,superoptimal,N,Y,0.000000,1.000000,0.000000,0.000000,-4.000000,0.000000,-2.000000
3,17.500000,10.000000,0.000000,-7.500000,0.000000,19.500000,0.000000,17.500000,N,N,-,-,N,N,-0.500000,0.000000,1.500000,0.000000,0.000000,-2.500000,0.000000
4,20.000000,0.000000,10.000000,0.000000,0.000000,-8.000000,15.000000,10.000000,N,N,-,-,N,N,2.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,-3.000000
5,20.000000,5.000000,0.000000,0.000000,5.000000,12.000000,0.000000,15.000000,Y,N,suboptimal,superoptimal,N,Y,0.000000,0.000000,1.333333,0.000000,0.000000,-2.000000,-0.333333
6,22.500000,0.000000,0.000000,7.500000,10.000000,4.500000,0.000000,12.500000,Y,N,suboptimal,superoptimal,N,Y,0.000000,0.000000,1.500000,0.000000,0.500000,-2.000000,0.000000
7,24.000000,-3.000000,0.000000,12.000000,13.000000,0.000000,0.000000,11.000000,N,N,-,-,N,N,0.000000,0.333333,1.333333,0.000000,0.000000,-1.333333,0.000000
8,27.000000,0.000000,2.250000,7.500000,7.750000,0.000000,0.000000,10.250000,Y,N,suboptimal,superoptimal,N,Y,0.000000,1.000000,1.000000,0.000000,-1.000000,0.000000,0.000000
9,28.000000,4.000000,6.000000,0.000000,0.000000,0.000000,3.000000,10.000000,Y,N,suboptimal,superoptimal,N,Y,4.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-4.000000
